In [99]:
use mavenfuzzyfactory

Commands completed successfully

Total execution time: 00:00:00

In [100]:
select * from website_pageviews limit 10

(10 row(s) affected)

Total execution time: 00:00:01.026

website_pageview_id,created_at,website_session_id,pageview_url
1,2012-03-19 08:04:16,1,/home
2,2012-03-19 08:16:49,2,/home
3,2012-03-19 08:26:55,3,/home
4,2012-03-19 08:37:33,4,/home
5,2012-03-19 09:00:55,5,/home
6,2012-03-19 09:05:46,6,/home
7,2012-03-19 09:06:27,7,/home
8,2012-03-19 09:10:08,6,/products
9,2012-03-19 09:10:52,6,/the-original-mr-fuzzy
10,2012-03-19 09:14:02,6,/cart


**Subject: Top Website Pages:** Could you help me get my head around the site by pulling <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">the most-viewed website pages, ranked by session volume?</span>

In [101]:
select pageview_url, count(DISTINCT website_session_id) as session
from website_pageviews
where created_at < '2012-06-09'
GROUP BY pageview_url
ORDER BY session desc
limit 10

(7 row(s) affected)

Total execution time: 00:00:01.397

pageview_url,session
/home,10403
/products,4239
/the-original-mr-fuzzy,3037
/cart,1306
/shipping,869
/billing,716
/thank-you-for-your-order,306


**Subject: Top Entry Pages -** Would you be able to pull a list of the top entry pages? I <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">want to confirm where our users are hitting the site.&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">If you could pull all entry pages and rank them on entry&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">volume, that would be great.</span>

In [102]:
-- creating a temporary table for first_entry_page later to be used.
CREATE TEMPORARY TABLE first_pageview
SELECT website_session_id, min(website_pageview_id) as first_entry_page
FROM website_pageviews
where created_at < '2012-06-12'
GROUP by website_session_id

Total execution time: 00:00:00.003

In [103]:
-- drop TEMPORARY table first_pageview

: 1050 (42S01): Table 'first_pageview' already exists

Total execution time: 00:00:00

In [21]:
select * from first_pageview limit 10

(10 row(s) affected)

Total execution time: 00:00:01.002

website_session_id,first_entry_page
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,12
9,14
10,15


In [22]:
-- IMPORTANT: 
-- LEFT TABLE will be the View we created it's logical to have info for these only
-- We have to join on pageviews it will contain info about more pageview to count session
select website_pageviews.pageview_url, count(DISTINCT first_pageview.website_session_id) as sessions_hitting_this_landing_page
from first_pageview LEFT JOIN website_pageviews
on website_pageviews.website_pageview_id = first_pageview.first_entry_page
where created_at < '2012-06-12'
GROUP BY pageview_url
order by 2 desc

(1 row(s) affected)

Total execution time: 00:00:01.037

pageview_url,sessions_hitting_this_landing_page
/home,10714


**Subject: Bounce Rate Analysis:** The other day you showed us that all of our traffic is landing <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">on the homepage right now. We should check how that&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">landing page is performing.</span>

Can you pull bounce rates for traffic landing on the <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">homepage? I would like to see three numbers...Sessions,&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Bounced Sessions, and % of Sessions which Bounced&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">(aka "Bounce Rate").</span>

### **BOUNCE RATE: The Concept for Bounce Rate Calculation is -- If the session has only one pageview which means the user only landed on one page and then existed the website ( Hence Bounced).**

In [23]:
-- BUSINESS CONTEXT: we want to see landing page performance for a certain time period (bounce rate)
-- STEP 1: find the first website_pageview_id for relevant sessions -- IMPORTANT for finding the first landing page.
-- STEP 2: identify the landing page of each session
-- STEP 3: counting pageviews for each session, to identify "bounces" -- IMPORTANT it's counting pageviews for each session later to identify bounce
-- STEP 4: summarizing total sessions and bounced sessions, by LP

Total execution time: 00:00:00

In [24]:
-- finding the first or minimum website pageview id associated with each session and storing in views
-- since we have to get the date from the specific table therefore the join on website session
create TEMPORARY table first_pageviews_demo
select website_sessions.website_session_id, 
min(website_pageviews.website_pageview_id) as first_pageview_id
from website_pageviews 
join website_sessions 
on website_pageviews.website_session_id = website_sessions.website_session_id
where website_sessions.created_at < '2012-06-14'
group by website_sessions.website_session_id

Commands completed successfully

Total execution time: 00:00:00.704

In [25]:
select * from first_pageviews_demo limit 10

(10 row(s) affected)

Total execution time: 00:00:01.005

website_session_id,first_pageview_id
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,12
9,14
10,15


In [26]:
-- landing page to each session -- On clause is for mapping sessions (but )
-- data on using session_id > first_pageview_id for joining -- because we have all the sessions but only selected pageview
-- first_pageview_demo has repeated pageview but no repeated session check by using csv
create TEMPORARY table sessions_w_landing_page_demo
select first_pageviews_demo.website_session_id, website_pageviews.pageview_url 
from first_pageviews_demo 
LEFT JOIN website_pageviews 
ON first_pageviews_demo.first_pageview_id = website_pageviews.website_pageview_id -- website pageview is the landing page

Commands completed successfully

Total execution time: 00:00:00.148

In [27]:
select * from sessions_w_landing_page_demo limit 10

(10 row(s) affected)

Total execution time: 00:00:01.010

website_session_id,pageview_url
1,/home
2,/home
3,/home
4,/home
5,/home
6,/home
7,/home
8,/home
9,/home
10,/home


In [28]:
-- table to include count of pageview per session
create TEMPORARY table bounced_sessions_only
select sessions_w_landing_page_demo.website_session_id, sessions_w_landing_page_demo.pageview_url, count(website_pageviews.website_pageview_id) as count_of_pages_viewed
from sessions_w_landing_page_demo
left join website_pageviews
on sessions_w_landing_page_demo.website_session_id = website_pageviews.website_session_id
GROUP BY 1, 2
having count(website_pageviews.website_pageview_id) = 1

Commands completed successfully

Total execution time: 00:00:00.081

In [42]:
select * from bounced_sessions_only limit 10

(10 row(s) affected)

Total execution time: 00:00:01.008

website_session_id,pageview_url,count_of_pages_viewed
1,/home,1
2,/home,1
3,/home,1
4,/home,1
5,/home,1
7,/home,1
8,/home,1
9,/home,1
10,/home,1
11,/home,1


In [29]:
SELECT
sessions_w_landing_page_demo.pageview_url,
count( DISTINCT sessions_w_landing_page_demo.website_session_id) as sessions,
count( DISTINCT bounced_sessions_only.website_session_id) AS bounced,
count( DISTINCT bounced_sessions_only.website_session_id)/count( DISTINCT sessions_w_landing_page_demo.website_session_id) as bounce_rate
FROM sessions_w_landing_page_demo
LEFT JOIN bounced_sessions_only
ON sessions_w_landing_page_demo.website_session_id = bounced_sessions_only.website_session_id
GROUP BY 1


(1 row(s) affected)

Total execution time: 00:00:01.026

pageview_url,sessions,bounced,bounce_rate
/home,11048,6538,0.5918


**Subject: Help Analyzing LP Test**

Based on your bounce rate analysis, we ran a new custom <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">landing page (/lander-1) in a 50/50 test against the&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">homepage (/home) for our gsearch nonbrand traffic.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Can you pull bounce rates for the two groups so we can&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">evaluate the new page? Make sure to just look at the time&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">period where /lander-1 was getting traffic, so that it is a fair&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">comparison.</span>

In [ ]:
-- STEP 0: find out when the new page /lander launched
-- STEP 1: finding the first website_pageview_id for relevant sessions
-- STEP 2: identifying the landing page of each session
-- STEP 3: counting pageviews for each session, to identify "bounces"
-- STEP 4: summarizing total sessions and bounced sessions, by LP

In [40]:
select min(website_pageview_id) as first_pageview_id, min(created_at) as first_created_at from website_pageviews where pageview_url = '/lander-1'

(1 row(s) affected)

Total execution time: 00:00:01.374

first_pageview_id,first_created_at
23504,2012-06-19 00:35:54


In [104]:
-- finding the first pageview for the each session with filters on date and others.
-- will use the first_pageview_id to limit the home page session time also.
create TEMPORARY table first_test_pageviews
select website_pageviews.website_session_id,
min(website_pageviews.website_pageview_id) as first_pageview
from website_pageviews
JOIN website_sessions
On website_pageviews.website_session_id = website_sessions.website_session_id
and website_sessions.created_at < '2012-07-28'
and website_pageviews.website_pageview_id > 23504
and utm_source = 'gsearch'
and utm_campaign = 'nonbrand'
GROUP by website_pageviews.website_session_id

Commands completed successfully

Total execution time: 00:00:00.704

In [107]:
select * from first_test_pageviews limit 10;

(10 row(s) affected)

Total execution time: 00:00:01.006

website_session_id,first_pageview
11684,23505
11685,23506
11686,23507
11687,23509
11688,23510
11689,23511
11690,23514
11691,23515
11692,23517
11693,23518


In [108]:
-- finding the first url for each pageview
-- since we have to find the first landing page for every pageview therefore on clause will have pageview_id rather than session_id
create TEMPORARY table nonbrand_test_sessions_w_landing_page
select first_test_pageviews.website_session_id, 
website_pageviews.pageview_url as landing_page
from first_test_pageviews
left join website_pageviews
on first_test_pageviews.first_pageview = website_pageviews.website_pageview_id
where website_pageviews.pageview_url in ('/home','/lander-1')

Commands completed successfully

Total execution time: 00:00:00.079

In [109]:
select * from nonbrand_test_sessions_w_landing_page limit 10

(10 row(s) affected)

Total execution time: 00:00:01.003

website_session_id,landing_page
11684,/home
11685,/lander-1
11686,/lander-1
11687,/home
11688,/home
11689,/lander-1
11690,/home
11691,/lander-1
11692,/lander-1
11693,/lander-1


In [110]:
-- finding count of pageviews for each session and filtering to 1 to know which session where bounce
create TEMPORARY table nonbrand_test_bounced_sessions
select nonbrand_test_sessions_w_landing_page.website_session_id, 
nonbrand_test_sessions_w_landing_page.landing_page, 
count(website_pageviews.website_pageview_id) as count_of_pages_viewed
from nonbrand_test_sessions_w_landing_page
left join website_pageviews
on nonbrand_test_sessions_w_landing_page.website_session_id = website_pageviews.website_session_id
GROUP BY 1, 2
having count(website_pageviews.website_pageview_id) = 1

Commands completed successfully

Total execution time: 00:00:00.041

In [111]:
select * from nonbrand_test_bounced_sessions limit 10

(10 row(s) affected)

Total execution time: 00:00:01.004

website_session_id,landing_page,count_of_pages_viewed
11684,/home,1
11685,/lander-1,1
11687,/home,1
11688,/home,1
11690,/home,1
11692,/lander-1,1
11696,/home,1
11697,/lander-1,1
11698,/lander-1,1
11701,/lander-1,1


In [112]:
SELECT
nonbrand_test_sessions_w_landing_page.landing_page,
count( DISTINCT nonbrand_test_sessions_w_landing_page.website_session_id) as sessions,
count( DISTINCT nonbrand_test_bounced_sessions.website_session_id) AS bounced,
count( DISTINCT nonbrand_test_bounced_sessions.website_session_id)/count( DISTINCT nonbrand_test_sessions_w_landing_page.website_session_id) as bounce_rate
FROM nonbrand_test_sessions_w_landing_page
LEFT JOIN nonbrand_test_bounced_sessions
ON nonbrand_test_sessions_w_landing_page.website_session_id = nonbrand_test_bounced_sessions.website_session_id
GROUP BY 1

(2 row(s) affected)

Total execution time: 00:00:01.032

landing_page,sessions,bounced,bounce_rate
/home,2261,1319,0.5834
/lander-1,2315,1232,0.5322


**Subject: Landing Page Trend Analysis**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you pull the volume of paid search nonbrand traffic&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">landing on /home and /lander-1, trended weekly since June&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">1st? I want to confirm the traffic is all routed correctly.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you also pull our overall paid search bounce rate&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">trended weekly? I want to make sure the lander change has&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">improved the overall picture.</span>

In [118]:
create TEMPORARY table sessions_w_first_pv_id_and_view_count
select website_pageviews.website_session_id,
min(website_pageviews.website_pageview_id) as first_pageview,
count(website_pageviews.website_pageview_id) as count_pageviews
from website_pageviews
JOIN website_sessions
On website_pageviews.website_session_id = website_sessions.website_session_id
and website_sessions.created_at > '2012-06-01'
and website_sessions.created_at < '2012-08-31'
and utm_source = 'gsearch'
and utm_campaign = 'nonbrand'
GROUP by website_pageviews.website_session_id

Commands completed successfully

Total execution time: 00:00:00.416

In [127]:
select * from sessions_w_first_pv_id_and_view_count limit 10

(10 row(s) affected)

Total execution time: 00:00:01.005

website_session_id,first_pageview,count_pageviews
9350,18598,3
9351,18600,3
9352,18601,4
9354,18611,1
9356,18616,6
9357,18622,1
9358,18623,3
9359,18626,1
9360,18627,1
9361,18628,1


In [124]:
CREATE TEMPORARY TABLE sessions_w_counts_lander_and_created_at
SELECT
sessions_w_first_pv_id_and_view_count.website_session_id,
sessions_w_first_pv_id_and_view_count.first_pageview,
sessions_w_first_pv_id_and_view_count.count_pageviews,
website_pageviews.pageview_url AS landing_page,
website_pageviews.created_at AS session_created_at
FROM sessions_w_first_pv_id_and_view_count
LEFT JOIN website_pageviews
ON sessions_w_first_pv_id_and_view_count.first_pageview = website_pageviews.website_pageview_id

Commands completed successfully

Total execution time: 00:00:00.113

In [126]:
select * from sessions_w_counts_lander_and_created_at limit 10

(10 row(s) affected)

Total execution time: 00:00:01.005

website_session_id,first_pageview,count_pageviews,landing_page,session_created_at
9350,18598,3,/home,2012-06-01 00:05:11
9351,18600,3,/home,2012-06-01 00:06:39
9352,18601,4,/home,2012-06-01 00:08:27
9354,18611,1,/home,2012-06-01 01:08:43
9356,18616,6,/home,2012-06-01 01:37:31
9357,18622,1,/home,2012-06-01 02:29:33
9358,18623,3,/home,2012-06-01 02:39:16
9359,18626,1,/home,2012-06-01 03:12:34
9360,18627,1,/home,2012-06-01 04:09:37
9361,18628,1,/home,2012-06-01 04:10:53


In [130]:
select 
min(DATE(session_created_at)),
COUNT(DISTINCT CASE WHEN count_pageviews = 1 THEN website_session_id ELSE NULL END)/ 
COUNT(DISTINCT website_session_id) AS bounce_rate,
count(DISTINCT case when landing_page='/home' then website_session_id else null end) as home_session,
count(DISTINCT case when landing_page='/lander-1' then website_session_id else null end) as lander_session
from sessions_w_counts_lander_and_created_at
GROUP by YEARWEEK(session_created_at)

(14 row(s) affected)

Total execution time: 00:00:01.114

min(DATE(session_created_at)),bounce_rate,home_session,lander_session
2012-06-01,0.6057,175,0
2012-06-03,0.5871,792,0
2012-06-10,0.6160,875,0
2012-06-17,0.5582,492,350
2012-06-24,0.5828,369,386
2012-07-01,0.5821,392,388
2012-07-08,0.5668,390,411
2012-07-15,0.5424,429,421
2012-07-22,0.5138,402,394
2012-07-29,0.4971,33,995
